In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

In [42]:
# Set up Chrome options
chrome_options = webdriver.ChromeOptions()

# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Go to Rightmove website
driver.get("https://www.rightmove.co.uk")
# Wait for the cookie popup to be present and then locate the 'Accept all' button
wait = WebDriverWait(driver, 10)
accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Accept all"]')))

# Click the 'Accept all' button
accept_button.click()

# Wait until the search input is present
search_input = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input.ksc_inputText.ksc_typeAheadInputField')))

# Clear the search box before entering text
search_input.clear()
search_input.send_keys("London")

# Wait for the autocomplete suggestions to appear and use the down arrow key to select the first suggestion
search_input.send_keys(Keys.DOWN)

# Use the ENTER key to select the top suggestion from the dropdown
search_input.send_keys(Keys.ENTER)

# Wait for the "To Rent" button that contains the text "To Rent" to be clickable
to_rent_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "To Rent")]')))
to_rent_button.click()

# Select the minimum price
min_price_select = wait.until(EC.presence_of_element_located((By.ID, 'minPrice')))
Select(min_price_select).select_by_value("2500")

# Select the maximum price
max_price_select = wait.until(EC.presence_of_element_located((By.ID, 'maxPrice')))
Select(max_price_select).select_by_value("3000")

# Select the minimum number of bedrooms
min_bedrooms_select = wait.until(EC.presence_of_element_located((By.ID, 'minBedrooms')))
Select(min_bedrooms_select).select_by_value("2")

# Select the maximum number of bedrooms
max_bedrooms_select = wait.until(EC.presence_of_element_located((By.ID, 'maxBedrooms')))
Select(max_bedrooms_select).select_by_value("3")

# Select the property type
property_type_select = wait.until(EC.presence_of_element_located((By.ID, 'displayPropertyType')))
Select(property_type_select).select_by_visible_text("Flats / Apartments")

# Wait for the "Find properties" button to be clickable
find_properties_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Find properties")]')))
find_properties_button.click()

In [43]:
# Initialize an empty list to store the URLs
property_urls = []

# Start processing pages
while True:
    # Wait for the properties to be loaded on the page
    properties = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.l-searchResult[data-test*="propertyCard"]'))
    )

    # Iterate over the properties
    for property in properties:
        try:
            # Check if the property has at least 2 bathrooms by looking for the icon's title
            bathroom_icon = property.find_element(By.CSS_SELECTOR, 'span.no-svg-bathroom-icon + span.text')
            num_bathrooms = int(bathroom_icon.get_attribute('textContent'))
            
            # If the property has at least 2 bathrooms, get the URL and add to the list
            if num_bathrooms >= 2:
                property_url_element = property.find_element(By.CSS_SELECTOR, 'a.propertyCard-link')
                property_url = property_url_element.get_attribute('href')
                property_urls.append(property_url)
        except NoSuchElementException:
            # If the bathroom element is not found, skip this property
            continue

    # Check if the "Next" button is disabled
    next_button = driver.find_element(By.CSS_SELECTOR, 'button.pagination-direction--next')
    if next_button.get_attribute('disabled'):
        # If the button is disabled, we are on the last page
        break
    else:
        # If the button is not disabled, click it to go to the next page
        ActionChains(driver).move_to_element(next_button).click().perform()


In [40]:
# Close the browser
driver.quit()

In [44]:
property_urls

['https://www.rightmove.co.uk/properties/142243013#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143162135#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143161874#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/141550130#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143161583#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143161526#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143161043#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/140462768#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143160653#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143160581#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143160545#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143159918#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143350943#/?channel=RES_LET',
 'https://www.rightmove.co.uk/properties/143158385#/?channel=RES_LET',
 'http

In [45]:
len(property_urls)

130